In [1]:
import pomegranate
import torch
import numpy as np
import copy
import random
import json
import pandas as pd
import collections
import time
import math
import itertools

In [2]:
import sys
sys.path.append('/Users/ziniuwu/Desktop/research/BayesNet')
from Structure.tools import discretize_series
from Testing.toy_dataset import *

In [27]:
class Single_BN():
    """
    Build a single Bayesian Network for a single table.
    Initialize with an appropriate table_name.
    """
    
    def __init__(self, table_name, method='Pome'):
        self.table_name = table_name
        self.n_in_bin = dict()
        self.encoding = dict()
        self.mapping = dict()
        self.max_value = dict()
        self.model = None
        self.method = method
        
        
    def build_discrete_table(self, data, n_mcv, n_bins, drop_na=True, ignore_cols = []):
        """
        Discretize the entire table use bining (This is using histogram method for continuous data)
        ::Param:: table: original table
                  n_mcv: for categorical data we keep the top n most common values and bin the rest
                  n_bins: number of bins for histogram, larger n_bins will provide more accuracy but less efficiency
                  drop_na: if True, we drop all rows with nan in it
                  ignore_cols: drop the unnessary columns for example id attribute
        """
        table = data.copy()
        if drop_na:
            table = table.dropna()
        for col in table.columns:
            if col in ignore_cols:
                table = table.drop(col, axis=1)
            else:
                table[col], self.n_in_bin[col], self.encoding[col], self.mapping[col] = discretize_series(
                    table[col],
                    n_mcv=n_mcv,
                    n_bins=n_bins,
                    drop_na= not drop_na
                )
                self.max_value[col] = int(table[col].max())+1
        self.node_names = list(table.columns)
        return table

    
    def apply_encoding_to_value(self, value, col):
        #return the encoded value given real value
        if col not in self.encoding:
            return value
        elif value not in self.encoding[col]:
            return value
        return self.encoding[col][value]
    
    
    def apply_ndistinct_to_value(self, enc_value, value, col):
        #return the number of distinct value in the bin
        if col not in self.n_in_bin:
            return 1
        elif enc_value not in self.n_in_bin[col]:
            return 1
        elif type(self.n_in_bin[col][enc_value])==int:
            return 1/self.n_in_bin[col][enc_value]
        elif value not in self.n_in_bin[col][enc_value]:
            return 1
        else:
            return self.n_in_bin[col][enc_value][value]
    
    def build_from_data(self, dataset, n_mcv=30, n_bins=60, ignore_cols=['id'], algorithm="greedy",
                        drop_na=True, max_parents=-1, root=0, n_jobs=8):
        """ Build the Pomegranate model from data, including structure learning and paramter learning
            ::Param:: dataset: pandas.dataframe
                      n_mcv: for categorical data we keep the top n most common values and bin the rest
                      n_bins: number of bins for histogram, larger n_bins will provide more accuracy but less efficiency
            for other parameters, pomegranate gives a detailed explaination:
            https://pomegranate.readthedocs.io/en/latest/BayesianNetwork.html
        """
        self.nrows = len(dataset)
        self.algorithm = algorithm
        self.max_parents = max_parents
        self.n_mcv = n_mcv
        self.n_bins = n_bins
        self.root = root
        
        discrete_table = self.build_discrete_table(dataset, n_mcv, n_bins, drop_na, ignore_cols)
        print(f'building pomegranate.BayesianNetwork from data with {self.nrows} rows')
        t = time.time()
        self.model = pomegranate.BayesianNetwork.from_samples(discrete_table,
                                                  algorithm=algorithm,
                                                  state_names=self.node_names,
                                                  max_parents=max_parents,
                                                  n_jobs=8,
                                                  root = self.root)
        print(f'Took {time.time() - t} secs.')
        
        

    def __str__(self):
        return f"bn{self.table_name}.{self.algorithm}-{self.max_parents}-{self.root}-{self.n_mcv}-{self.n_bins}"
    
    
    def load(self, path, pgm_path=None):
        with open(path, 'r') as myfile:
            json_model = myfile.read()
        self.model = BayesianNetwork.from_json(json_model)
            
            
    def save(self, path, pgm_path=None):
        with open(path, 'w') as outfile:
            outfile.write(self.model.to_json())
            
    
    def loopy_belief_propagation(self, evidence, n_distinct):
        """Performance a LBP in random order.
           This adapts the LBP implemented in pomegranate package itself.
        """
        index = list(range(len(self.node_names)))
        p_estimate = 1
        
        while len(index)!=0:
            i = random.choice(index)
            val = evidence[i]
            if val is not None:
                evidence[i] = None
                dist = self.model.predict_proba(evidence)
                p = dist[i].parameters[0][val]*n_distinct[i]
                p_estimate *= p
            index.remove(i)
        return p_estimate
        
            
    def infer_point_query_LBP(self, query, num_samples=1, return_prob=False):
        """Probability inference using Loopy belief propagation. For example estimate P(X=x, Y=y, Z=z)
           ::Param:: query: dictionary of the form {X:x, Y:y, Z:z}
                     x,y,z can only be a single value
                     num_samples: how many times to run inference. Since Loopy belief propagation is sometime
                     an approaximation, we might to run it for multiple times and take the average.
                     return_prob: if true, return P(X=x, Y=y, Z=z)
                                  else return P(X=x, Y=y, Z=z)*nrows
        """
        ncols = len(query)
        nrows = self.nrows
                    
        evidence = [None]*len(self.node_names)
        n_distinct = [1]*len(self.node_names)
        for attr in query:
            ind = self.node_names.index(attr)
            evidence[ind] = self.apply_encoding_to_value(query[attr], attr)
            n_distinct[ind] = self.apply_ndistinct_to_value(evidence[ind], query[attr], attr)
        
        if num_samples == 1:
            #Using topological order to infer probability
            sampling_order = []
            while len(sampling_order) < len(self.model.structure):
                for i, deps in enumerate(self.model.structure):
                    if i in sampling_order:
                        continue  # already ordered
                    if all(d in sampling_order for d in deps):
                        sampling_order.append(i)
            
            p_estimate = 1
            for i in sampling_order:
                val = evidence[i]
                if val is not None:
                    evidence[i] = None
                    dist = self.model.predict_proba(evidence)
                    p = dist[i].parameters[0][val]*n_distinct[i]
                    p_estimate *= p
                        
        else:
            p_estimates = []
            for i in range(num_samples):
                copy_evidence = copy.deepcopy(evidence)
                p_estimates.append(self.loopy_belief_propagation(copy_evidence, n_distinct))
            p_estimate = sum(p_estimates)/num_samples
        
        if return_prob:
            return (p_estimate, nrows)
        return int(p_estimate * nrows)
    
    
    def infer_range_query_LBP(self, query, num_samples=1, return_prob=False):
        """Probability inference using Loopy belief propagation. For example estimate P(X=x, Y=y, Z=z)
           ::Param:: query: dictionary of the form {X:[x], Y:[y], Z:[z]}
                     x,y,z can only be set of single value
                     num_samples: how many times to run inference. Since Loopy belief propagation is sometime
                     an approaximation, we might to run it for multiple times and take the average.
                     return_prob: if true, return P(X=x, Y=y, Z=z)
                                  else return P(X=x, Y=y, Z=z)*nrows
           LBP for estimating range query can be really slow
        """
        def cartesian_product(d):
            target_list = []
            for key in d:
                val = d[key]
                if type(val) != list:
                    val = [val]
                target_list.append(val)
            return itertools.product(*target_list)
        
        
        p_estimate = 0
        for query_tuple in cartesian_product(query):
            point_query = dict()
            i = 0
            for attr in query:
                point_query[attr] = query_tuple[i]
                i += 1
            p_estimate += self.infer_point_query_LBP(point_query, return_prob=True)[0]
        
        return p_estimate*self.nrows
            

In [35]:
df = toy_data_highly_correlated_cat(nrows=100000, return_df=True)
df.head(20)

,id,cont_attr1,cont_attr2,cont_attr3,cont_attr4,cont_attr5,cont_attr6,cont_attr7,cont_attr8
0,0,5,5,10,10,20,15,15,50
1,1,0,0,0,0,0,0,10,10
2,2,3,3,6,6,12,8,13,34
3,3,3,4,7,9,16,15,13,41
4,4,7,7,14,14,28,17,17,66
5,5,9,9,18,18,36,22,19,82
6,6,3,4,7,7,14,17,13,37
7,7,5,5,10,10,20,14,15,50
8,8,2,2,4,4,8,11,12,26
9,9,4,4,8,8,16,17,14,42


In [36]:
BN = Single_BN('title')
BN.build_from_data(df, algorithm='chow-liu')

building pomegranate.BayesianNetwork from data with 100000 rows
Took 1.4268620014190674 secs.


In [37]:
BN.model.structure

((), (2,), (0,), (7,), (7,), (2,), (0,), (2,))

In [19]:
def debug(evidence, BN):
    pred = BN.model.predict_proba([None, None, None, None, None])
    p = 1
    for (col, v) in evidence:
        i = BN.node_names.index(col)
        print(pred[i].parameters[0][BN.encoding[col][v]])
        p*=pred[i].parameters[0][BN.encoding[col][v]]
    return p

In [38]:
tic = time.time()
print(len(df.query('cont_attr1 in [3,6,7]').query('cont_attr2 == 3').query('cont_attr4 == [6,7,14]')))
toc = time.time()
print(toc-tic)
print(BN.infer_range_query_LBP({'cont_attr1': [3,6,7], 'cont_attr2': 3, 'cont_attr4': [6,7,14]}, num_samples=1))
print(time.time()-toc)

8241
0.02106332778930664
8240.999999999889
2.7123477458953857


In [ ]:
len(df.query('production_year in [2011,2012]').query('random1 == 2').query('random2 == [10,11]'))

In [ ]:
print(BN.infer_range_query_LBP({'production_year': [2011,2012], 'random1':2, 'random2':[10,11]}, num_samples=1))

In [ ]:
BN.model.structure